In [48]:
import os, sys, inspect
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
# add parent dir to system dir
currdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootdir = os.path.dirname(currdir)
sys.path.insert(0, rootdir)

In [49]:
def interp1d(xnew, x, y, kind):
    f = interpolate.interp1d(x, y, kind="linear")
#     print(x, xnew)
    return f(xnew)

In [50]:
year = 2011
fname = f"household_income_{year}.csv"
ffname = os.path.join(rootdir, "data", "raw", fname)

income_df_ = pd.read_csv(ffname, skiprows=1)
income_df_ = income_df_[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_df_ = income_df_.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_df_ = income_df_[income_df_["Median Income (dollars)"] != '-']
income_df_["Year"] = year
income_df = income_df_

year = 2012
fname = f"household_income_{year}.csv"
ffname = os.path.join(rootdir, "data", "raw", fname)

income_df_ = pd.read_csv(ffname, skiprows=1)
income_df_ = income_df_[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_df_ = income_df_.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_df_ = income_df_[income_df_["Median Income (dollars)"] != '-']
income_df_["Year"] = year
income_df = income_df.append(income_df_, ignore_index=True)

year = 2013
fname = f"household_income_{year}.csv"
ffname = os.path.join(rootdir, "data", "raw", fname)

income_df_ = pd.read_csv(ffname, skiprows=1)
income_df_ = income_df_[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_df_ = income_df_.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_df_ = income_df_[income_df_["Median Income (dollars)"] != '-']
income_df_["Year"] = year
income_df = income_df.append(income_df_, ignore_index=True)

year = 2014
fname = f"household_income_{year}.csv"
ffname = os.path.join(rootdir, "data", "raw", fname)

income_df_ = pd.read_csv(ffname, skiprows=1)
income_df_ = income_df_[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_df_ = income_df_.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_df_ = income_df_[income_df_["Median Income (dollars)"] != '-']
income_df_["Year"] = year
income_df = income_df.append(income_df_, ignore_index=True)

year = 2015
fname = f"household_income_{year}.csv"
ffname = os.path.join(rootdir, "data", "raw", fname)

income_df_ = pd.read_csv(ffname, skiprows=1)
income_df_ = income_df_[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_df_ = income_df_.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_df_ = income_df_[income_df_["Median Income (dollars)"] != '-']
income_df_["Year"] = year
income_df = income_df.append(income_df_, ignore_index=True)

year = 2016
fname = f"household_income_{year}.csv"
ffname = os.path.join(rootdir, "data", "raw", fname)

income_df_ = pd.read_csv(ffname, skiprows=1)
income_df_ = income_df_[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_df_ = income_df_.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_df_ = income_df_[income_df_["Median Income (dollars)"] != '-']
income_df_["Year"] = year
income_df = income_df.append(income_df_, ignore_index=True)


In [51]:
# perform data cleansing
income_df["Number of Households"] = income_df["Number of Households"].apply(lambda x: int(x))
income_df["Median Income (dollars)"].loc[income_df["Median Income (dollars)"] == '2,500-'] = 25000
income_df["Median Income (dollars)"].loc[income_df["Median Income (dollars)"] == '250,000+'] = 25000
income_df = income_df.loc[income_df["Median Income (dollars)"] != "(X)"]
income_df["Median Income (dollars)"] = income_df["Median Income (dollars)"].apply(pd.to_numeric)
income_df["Year"] = income_df["Year"].apply(pd.to_numeric)

D:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [52]:
# perform interplation to fill month values
interp_kind = "slinear"
income_interp_df = pd.DataFrame()
for zipcode in income_df["Zip Code"].unique():
    # get zip code data frame
    zipcode_df = income_df.loc[income_df["Zip Code"] == zipcode, :]
    
    # get values
    n_years = zipcode_df.shape[0]
    years = zipcode_df["Year"].values
    incomes = zipcode_df["Median Income (dollars)"].values
    households = zipcode_df["Number of Households"].values
    
    # get interpolations
    x_interp = np.linspace(1, n_years * 12,  n_years * 12)
    x = np.insert(np.linspace(1, n_years,  n_years) * 12, 0, 0)
    
    incomes_interp = interp1d(
        x_interp, x, np.insert(incomes, 0, incomes[0]), kind=interp_kind).astype(float).tolist()
    households_interp = interp1d(
        x_interp, x, np.insert(households, 0, households[0]), kind=interp_kind).astype(int).tolist()
    years_interp = np.repeat(years, 12).tolist()
    month_interp = np.tile(np.linspace(1, 12, 12).astype(int), n_years).tolist()
    
    
    # create new
    zipcode_interp_df = pd.DataFrame({"Year": years_interp,
                                      "Month": month_interp,
                                      "Number of Households": households_interp,
                                      "Median Income (dollars)": incomes_interp,
                                      "Zip Code": zipcode})
    income_interp_df = income_interp_df.append(zipcode_interp_df)

output = os.path.join(rootdir, "data", "int", "median_income_by_zip.csv")
income_interp_df.to_csv(output, index=False)
income_interp_df

,Median Income (dollars),Month,Number of Households,Year,Zip Code
0,42009.00,1,605,2011,73949
1,42009.00,2,605,2011,73949
2,42009.00,3,605,2011,73949
3,42009.00,4,605,2011,73949
4,42009.00,5,605,2011,73949
5,42009.00,6,605,2011,73949
6,42009.00,7,605,2011,73949
7,42009.00,8,605,2011,73949
8,42009.00,9,605,2011,73949
9,42009.00,10,605,2011,73949
